In [59]:
import pandas as pd
import altair as alt
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

import statsmodels.api as sm

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

import warnings
warnings.filterwarnings('ignore')

alt.renderers.enable('html')

RendererRegistry.enable('html')

In [ ]:
FanBa = pd.read_excel('BOLT UBC First Byte - Fanbase Engagement.xlsx', engine="openpyxl")
print("FanBa:", FanBa.shape)
FanBa.info()


Merch = pd.read_excel('BOLT UBC First Byte - Merchandise Sales.xlsx', engine="openpyxl")
print("Merch:", Merch.shape)
Merch.info()


Stadi = pd.read_excel('BOLT UBC First Byte - Stadium Operations.xlsx', engine="openpyxl")
print("Stadi:", Stadi.shape)
Stadi.info()

FanBa: (70000, 5)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 70000 entries, 0 to 69999
Data columns (total 5 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Membership_ID    70000 non-null  int64 
 1   Age_Group        70000 non-null  object
 2   Games_Attended   70000 non-null  int64 
 3   Seasonal_Pass    70000 non-null  bool  
 4   Customer_Region  70000 non-null  object
dtypes: bool(1), int64(2), object(2)
memory usage: 2.2+ MB
Merch: (67877, 13)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 67877 entries, 0 to 67876
Data columns (total 13 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   Product_ID          67877 non-null  int64         
 1   Barcode             67877 non-null  int64         
 2   Item_Category       67877 non-null  object        
 3   Item_Name           67877 non-null  object        
 4   Size                52041 non-n

In [ ]:
def standardize_columns(df):
    df = df.copy()
    df.columns = (
        df.columns
        .str.strip()
        .str.replace(r"[^\w]+", "_", regex=True)
        .str.lower()
    )
    return df

def strip_lower(series):
    return series.astype(str).str.strip().str.lower()

def to_bool_int(series):
    s = series.astype(str).str.strip().str.lower()
    true_vals = {"true", "1", "yes", "y", "t"}
    return s.isin(true_vals).astype(int)

def parse_date(series):
    return pd.to_datetime(series, errors="coerce", infer_datetime_format=True)

def winsorize(s, lo=0.01, hi=0.99):
    ql, qh = s.quantile(lo), s.quantile(hi)
    return s.clip(ql, qh)

def report_na(df, name):
    na_cnt = df.isna().sum()
    print(f"\n[缺失值汇总] {name}\n", na_cnt[na_cnt>0].sort_values(ascending=False))

def report_basic(df, name):
    print(f"\n=== {name} 基础信息 ===")
    print(df.shape)
    print(df.dtypes)
    print(df.head(3))

def corr_to_long_format(corr_df):
    corr_long = corr_df.reset_index().melt(id_vars='index')
    corr_long.columns = ['feature_x', 'feature_y', 'correlation']
    return corr_long

In [ ]:
# ===== Fanbase Clean =====
FanBa_raw = FanBa.copy()
FanBa = standardize_columns(FanBa_raw)

# 列名映射（容错）
col_map = {
    "membership_id": ["membership_id","member_id","memberid"],
    "age_group": ["age_group","age"],
    "games_attended": ["games_attended","games_attended_2024","games_2024","attended"],
    "seasonal_pass": ["seasonal_pass","season_pass","pass"],
    "customer_region": ["customer_region","region","fan_region"]
}
for std, aliases in col_map.items():
    for a in aliases:
        if a in FanBa.columns and std not in FanBa.columns:
            FanBa.rename(columns={a: a if std in FanBa.columns else a}, inplace=True)
            FanBa.rename(columns={a: std}, inplace=True)

# 类型/标准化
if "age_group" in FanBa.columns:       FanBa["age_group"] = strip_lower(FanBa["age_group"])
if "customer_region" in FanBa.columns: FanBa["customer_region"] = strip_lower(FanBa["customer_region"])
if "seasonal_pass" in FanBa.columns:
    FanBa["seasonal_pass"] = (FanBa["seasonal_pass"].astype(int)
                              if FanBa["seasonal_pass"].dtype == bool
                              else to_bool_int(FanBa["seasonal_pass"]))
if "games_attended" in FanBa.columns:
    FanBa["games_attended"] = pd.to_numeric(FanBa["games_attended"], errors="coerce")
    FanBa.loc[FanBa["games_attended"] < 0, "games_attended"] = np.nan

# 缺失/去重
if "membership_id" in FanBa.columns:
    FanBa = FanBa[~FanBa["membership_id"].isna()]
for c in ["age_group","customer_region"]:
    if c in FanBa.columns: FanBa[c] = FanBa[c].fillna("unknown")
if "games_attended" in FanBa.columns:
    FanBa["games_attended"] = FanBa["games_attended"].fillna(FanBa["games_attended"].median())
FanBa = FanBa.drop_duplicates(subset=["membership_id"]) if "membership_id" in FanBa.columns else FanBa.drop_duplicates()


FanBa_viz = FanBa.copy()
FanBa_viz["games_attended_w"] = winsorize(FanBa_viz["games_attended"]) if "games_attended" in FanBa_viz else None


FanBa_model = FanBa.copy()
cat_cols = [c for c in ["age_group","customer_region"] if c in FanBa_model.columns]
if cat_cols:
    FanBa_model = pd.get_dummies(FanBa_model, columns=cat_cols, drop_first=True)
if "games_attended" in FanBa_model:
    FanBa_model["games_attended_w"] = winsorize(FanBa_model["games_attended"])


report_basic(FanBa_viz, "FanBa_viz")
report_na(FanBa_viz, "FanBa_viz")

report_basic(FanBa_model, "FanBa_model")
report_na(FanBa_model, "FanBa_model")


=== FanBa_viz 基础信息 ===
(70000, 6)
membership_id         int64
age_group            object
games_attended      float64
seasonal_pass         int64
customer_region      object
games_attended_w    float64
dtype: object
   membership_id age_group  games_attended  seasonal_pass customer_region  \
0       10218708     18-25             3.0              0          canada   
1       10043567     18-25             2.0              0          canada   
2       14272475     41-60             2.0              0          canada   

   games_attended_w  
0               3.0  
1               2.0  
2               2.0  

[缺失值汇总] FanBa_viz
 Series([], dtype: int64)

=== FanBa_model 基础信息 ===
(70000, 14)
membership_id                    int64
games_attended                 float64
seasonal_pass                    int64
age_group_26-40                   bool
age_group_41-60                   bool
age_group_60+                     bool
age_group_<18                     bool
customer_region_china         

In [85]:
fan_corr = FanBa_model.corr(numeric_only=True)
fan_corr_long = corr_to_long_format(fan_corr)

fan_corr_long = fan_corr_long[
    ~fan_corr_long['feature_x'].str.contains('_w') &
    ~fan_corr_long['feature_y'].str.contains('_w')
]

fan_heatmap = alt.Chart(fan_corr_long).mark_rect().encode(
    x=alt.X('feature_x:N', sort=None, title='Variables'),
    y=alt.Y('feature_y:N', sort=None, title='Variables'),
    color=alt.Color('correlation:Q', scale=alt.Scale(scheme = 'blues')),
    tooltip=['feature_x', 'feature_y', 'correlation']
).properties(
    title="Fanbase Correlation Heatmap",
    width=400,
    height=400
)
fan_heatmap.display()

alt.Chart(...)

In [ ]:
# ===== Merchandise Clean =====
Merch_raw = Merch.copy()
Merch = standardize_columns(Merch_raw)

# 列名映射
col_map = {
    "product_id": ["product_id","pid","sku"],
    "barcode": ["barcode","upc","ean"],
    "item_category": ["item_category","category"],
    "item_name": ["item_name","name","item"],
    "size": ["size"],
    "unit_price": ["unit_price","price","unitprice"],
    "customer_age_group": ["customer_age_group","age_group"],
    "customer_region": ["customer_region","region"],
    "promotion": ["promotion","promo","discount","on_sale"],
    "channel": ["channel","sales_channel"],
    "selling_date": ["selling_date","order_date","date","sold_at"],
    "member_id": ["member_id","membership_id","customer_id"],
    "arrival_date": ["arrival_date","delivered_date","arrive_date"],
    "quantity": ["quantity","qty","units","count"]
}
for std, aliases in col_map.items():
    for a in aliases:
        if a in Merch.columns and std not in Merch.columns:
            Merch.rename(columns={a: std}, inplace=True)

# 基础类型/标准化
for c in ["item_category","item_name","size","customer_age_group","customer_region","channel"]:
    if c in Merch.columns: Merch[c] = strip_lower(Merch[c])
if "promotion" in Merch.columns:
    Merch["promotion"] = Merch["promotion"].astype(int) if Merch["promotion"].dtype == bool else to_bool_int(Merch["promotion"])
Merch["quantity"]   = pd.to_numeric(Merch.get("quantity", 1), errors="coerce").fillna(1)
Merch["unit_price"] = pd.to_numeric(Merch.get("unit_price", np.nan), errors="coerce")
for d in ["selling_date","arrival_date"]:
    if d in Merch.columns: Merch[d] = parse_date(Merch[d])

# 逻辑约束
Merch.loc[Merch["quantity"] <= 0, "quantity"] = np.nan
Merch.loc[Merch["unit_price"] < 0, "unit_price"] = np.nan

# 缺失/去重
if "product_id" in Merch.columns: Merch = Merch[~Merch["product_id"].isna()]
for c in ["item_category","item_name","size","customer_age_group","customer_region","channel"]:
    if c in Merch.columns: Merch[c] = Merch[c].fillna("unknown")
for n in ["quantity","unit_price"]:
    if n in Merch.columns: Merch[n] = Merch[n].fillna(Merch[n].median())
if "selling_date" in Merch.columns:
    subset_keys = ["product_id","selling_date"] + (["member_id"] if "member_id" in Merch.columns else [])
    Merch = Merch.drop_duplicates(subset=subset_keys)
else:
    Merch = Merch.drop_duplicates()

# 衍生列
Merch["revenue"]   = Merch["unit_price"] * Merch["quantity"]
Merch["selling_date"] = parse_date(Merch["selling_date"])
Merch = Merch.dropna(subset=["selling_date"])  # 后续按月绘图需要
Merch["month"]     = Merch["selling_date"].dt.month
Merch["year"]      = Merch["selling_date"].dt.year
Merch["revenue_w"] = winsorize(Merch["revenue"])

Merch_viz = Merch.copy()



Merch_model = Merch.copy()
cat_cols = [c for c in ["item_category","size","customer_age_group","customer_region","channel"] if c in Merch_model.columns]
if cat_cols:
    Merch_model = pd.get_dummies(Merch_model, columns=cat_cols, drop_first=True)
# 补充一些时间特征（可选）
Merch_model["is_promo"] = Merch_model.get("promotion", 0)


report_basic(Merch_viz, "Merch_viz")
report_na(Merch_viz, "Merch_viz")

report_basic(Merch_model, "Merch_model")
report_na(Merch_model, "Merch_model")


=== Merch_viz 基础信息 ===
(67873, 19)
product_id                          int64
barcode                             int64
item_category                      object
item_name                          object
size                               object
unit_price                        float64
customer_age_group                 object
customer_region                    object
promotion                           int64
channel                            object
selling_date               datetime64[ns]
member_id                           int64
arrival_date               datetime64[ns]
quantity                          float64
arrival_after_sell_flag             int64
revenue                           float64
month                               int32
year                                int32
revenue_w                         float64
dtype: object
   product_id   barcode item_category    item_name size  unit_price  \
0    10000000  20000000           cap  classic cap  nan        35.0   
1    10000

In [89]:
merch_corr = Merch_model.corr(numeric_only=True)
merch_corr_long = corr_to_long_format(merch_corr)

merch_corr_long = merch_corr_long[
    ~merch_corr_long['feature_x'].str.contains('_w') &
    ~merch_corr_long['feature_y'].str.contains('_w')
]



merch_heatmap = alt.Chart(merch_corr_long).mark_rect().encode(
    x=alt.X('feature_x:N', sort=None),
    y=alt.Y('feature_y:N', sort=None),
    color=alt.Color('correlation:Q', scale=alt.Scale(scheme='reds')),
    tooltip=['feature_x', 'feature_y', 'correlation']
).properties(
    title="Merchandise Correlation Heatmap",
    width=400,
    height=400
)
merch_heatmap.display()

alt.Chart(...)

In [ ]:
# ===== Stadium Clean =====
Stadi_raw = Stadi.copy()
Stadi = standardize_columns(Stadi_raw)

# 列名映射
col_map = {
    "month": ["month","mon"],
    "source": ["source","revenue_source","stream"],
    "revenue": ["revenue","amount","value","income"]
}
for std, aliases in col_map.items():
    for a in aliases:
        if a in Stadi.columns and std not in Stadi.columns:
            Stadi.rename(columns={a: std}, inplace=True)

# 类型/标准化
if "source" in Stadi.columns:  Stadi["source"] = strip_lower(Stadi["source"])
Stadi["month"]   = pd.to_numeric(Stadi.get("month", np.nan), errors="coerce")
Stadi["revenue"] = pd.to_numeric(Stadi.get("revenue", np.nan), errors="coerce")

# 合法值与缺失
Stadi.loc[~Stadi["month"].between(1,12, inclusive="both"), "month"] = np.nan
for c in ["source"]:
    if c in Stadi.columns: Stadi[c] = Stadi[c].fillna("unknown")
Stadi["month"]   = Stadi["month"].fillna(Stadi["month"].mode().iloc[0] if Stadi["month"].notna().any() else 1)
Stadi["revenue"] = Stadi["revenue"].fillna(Stadi["revenue"].median())
Stadi.loc[Stadi["revenue"] < 0, "revenue"] = np.nan
Stadi["revenue"] = Stadi["revenue"].fillna(Stadi["revenue"].median())

# 去重
if all(c in Stadi.columns for c in ["month","source"]):
    Stadi = Stadi.drop_duplicates(subset=["month","source"])
else:
    Stadi = Stadi.drop_duplicates()

Stadi["revenue_w"] = winsorize(Stadi["revenue"])


Stadi_viz = Stadi.copy()

Stadi_model = pd.get_dummies(Stadi, columns=["source"], drop_first=True)

report_basic(Stadi_viz, "Stadi_viz")
report_na(Stadi_viz, "Stadi_viz")

report_basic(Stadi_model, "Stadi_model")
report_na(Stadi_model, "Stadi_model")


=== Stadi_viz 基础信息 ===
(144, 4)
month        float64
source        object
revenue      float64
revenue_w    float64
dtype: object
   month      source   revenue  revenue_w
0    1.0  upper bowl   25020.0    25020.0
1    2.0  upper bowl  686664.0   686664.0
2    3.0  upper bowl  434112.0   434112.0

[缺失值汇总] Stadi_viz
 Series([], dtype: int64)

=== Stadi_model 基础信息 ===
(144, 14)
month                 float64
revenue               float64
revenue_w             float64
source_concert           bool
source_conference        bool
source_food              bool
source_insurance         bool
source_lower bowl        bool
source_maintenance       bool
source_premium           bool
source_season            bool
source_staff             bool
source_upper bowl        bool
source_utilities         bool
dtype: object
   month   revenue  revenue_w  source_concert  source_conference  source_food  \
0    1.0   25020.0    25020.0           False              False        False   
1    2.0  686664.0   686

In [92]:
stadi_corr = Stadi_model.corr(numeric_only=True)
stadi_corr_long = corr_to_long_format(stadi_corr)

stadi_corr_long = stadi_corr_long[
    ~stadi_corr_long['feature_x'].str.contains('_w') &
    ~stadi_corr_long['feature_y'].str.contains('_w')
]

stadi_heatmap = alt.Chart(stadi_corr_long).mark_rect().encode(
    x=alt.X('feature_x:N', sort=None),
    y=alt.Y('feature_y:N', sort=None),
    color=alt.Color('correlation:Q', scale=alt.Scale(scheme='greens')),
    tooltip=['feature_x', 'feature_y', 'correlation']
).properties(
    title="Stadium Revenue Correlation Heatmap",
    width=400,
    height=400
)
stadi_heatmap.display()

alt.Chart(...)